## Imports

In [25]:
# !pip install pandas
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import lava.lib.dl.slayer as slayer

import typing as ty
import numpy as np
import matplotlib.pyplot as plt
import logging
from PIL import Image

from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps, RunContinuous
from lava.proc.io.sink import RingBuffer as ReceiveProcess
from lava.proc.io.source import RingBuffer as SendProcess
from lava.proc import io

from lava.lib.dl import netx

## Original slayer inference evaluation

In [26]:
# Slayer network model that was used to train IRIS model
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        neuron_params = {
                'threshold'     : 1.75,
                'current_decay' : 0.25 , # this must be 1 to use batchnorm
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : False,
            }
        
        self.blocks = torch.nn.ModuleList([
                slayer.block.cuba.Dense(neuron_params, 4, 24, weight_norm=True),
                slayer.block.cuba.Dense(neuron_params, 24, 3, weight_norm=True),
            ])

    def forward(self, x):
        # Assume the tensor is in format NCT        
        for block in self.blocks:
            x = block(x)
        return x
    
# IRIS dataset format used to train slayer based model
class IrisDatasetSlayer(Dataset):
    def __init__(self, data_file, label_file, transform=None, target_transform=None, time_steps=8):
        features = pd.read_csv(data_file, header=None).values
        labels = pd.read_csv(label_file, header=None).values
        
        self.X = torch.tensor(features).type(torch.FloatTensor)
        self.X = self.X.reshape(150, 4, 1).repeat(1, 1, time_steps)
        self.y = torch.tensor(labels).squeeze(1)
        
        self.samples = features.shape[0]
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
        
    def __len__(self):
        return self.samples
    
# IRIS dataset infos
d = IrisDatasetSlayer('data/iris_data.csv', 'data/iris_label.csv')
print('Type X', type(d[0][0]))
print('Type y', type(d[0][1]))
print('Shape X', d[0][0].shape)
print('Shape y', d[0][1].shape)

Type X <class 'torch.Tensor'>
Type y <class 'torch.Tensor'>
Shape X torch.Size([4, 8])
Shape y torch.Size([])


In [27]:
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler

# Setup dataset
batch_size = 16
test_split = 0.2
shuffle_dataset = True
random_seed = 42

dataset = IrisDatasetSlayer('data/iris_data.csv', 'data/iris_label.csv')
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split*dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, test_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
# train_sampler = SequentialSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Setup dataloader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

# Setup network device
# device = torch.device('cpu')
device = torch.device('cuda') 
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

In [28]:
net = Network().to(device)
net_pt = torch.load('Trained/network.pt')
net.load_state_dict(net_pt)

error = slayer.loss.SpikeMax(mode='softmax').to(device)
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)

for epoch in range(1):
    for i, (input, label) in enumerate(test_loader): # test set
        output = assistant.test(input, label)
    print(f'\r[Epoch {epoch:2d}/{1}] {stats}')
    print('Correct samples (test set): ',stats.testing.correct_samples,'/',stats.testing.num_samples)
    
for epoch in range(1):
    for i, (input, label) in enumerate(train_loader): # train set
        output = assistant.test(input, label)
    print(f'\r[Epoch {epoch:2d}/{1}] {stats}')
    print('Correct samples (full set): ',stats.testing.correct_samples,'/',stats.testing.num_samples)

[Epoch  0/1] Train  | Test  loss =    12.62833                        accuracy = 1.00000
Correct samples (test set):  30 / 30
[Epoch  0/1] Train  | Test  loss =    12.91690                        accuracy = 0.95333
Correct samples (full set):  143 / 150


## Setup network, dataloaders, resetters, and loggers

In [29]:
net = netx.hdf5.Network(net_config='Trained' + '/network.net')

print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.__class__.__name__:5s} : {l.name:10s}, shape : {l.shape}')

[24]
[3]
There are 2 layers in network:
Dense : Process_8 , shape : (24,)
Dense : Process_11, shape : (3,)


In [30]:
from lava.proc import io

class IrisDatasetLava(Dataset):
    def __init__(self, data_file, label_file, transform=None, target_transform=None, time_steps=8):
        features = pd.read_csv(data_file, header=None).values
        labels = pd.read_csv(label_file, header=None).values
        
        self.X = torch.tensor(features).type(torch.FloatTensor).unsqueeze(2)
        self.X = self.X.reshape(150, 4, 1).repeat(1, 1, time_steps).numpy()
        self.y = torch.tensor(labels).squeeze(1).numpy()

        self.samples = features.shape[0]
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
        
    def __len__(self):
        return self.samples

# Dataset infos
d = IrisDatasetLava('data/iris_data.csv', 'data/iris_label.csv')
print('Type X', type(d[0][0]))
print('Type y', type(d[0][1]))
print('Shape X', d[0][0].shape)
print('Shape y', d[0][1].shape)

num_samples = 150
steps_per_sample = 8
readout_offset = (steps_per_sample-1) + len(net.layers)
num_steps = num_samples*steps_per_sample

full_set = IrisDatasetLava('data/iris_data.csv', 'data/iris_label.csv')

dataloader = io.dataloader.SpikeDataloader(
    dataset=full_set,
    interval=steps_per_sample,
)

net = netx.hdf5.Network(net_config='Trained' + '/network.net')

Type X <class 'numpy.ndarray'>
Type y <class 'numpy.int64'>
Shape X (4, 8)
Shape y ()
[24]
[3]


In [31]:
gt_logger = io.sink.RingBuffer(shape=(1,), buffer=num_steps)
output_logger = io.sink.Read(
    num_samples,
    interval=steps_per_sample,
    offset=readout_offset
)

# reset after every sample has been run
for i, l in enumerate(net.layers):
    u_resetter = io.reset.Reset(interval=steps_per_sample, offset=i)
    v_resetter = io.reset.Reset(interval=steps_per_sample, offset=i)
    u_resetter.connect_var(l.neuron.u)
    v_resetter.connect_var(l.neuron.v)

dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(net.in_layer.inp)

output_logger = io.sink.RingBuffer(shape=net.out_layer.shape, buffer=num_steps)
net.out_layer.out.connect(output_logger.a_in)

## Configure/run network for inference on Loihi1

In [32]:
class CustomRunConfig(Loihi1SimCfg):
    def select(self, proc, proc_models):
        # customize run config to always use float model for io.sink.RingBuffer
        if isinstance(proc, io.sink.RingBuffer):
            return io.sink.PyReceiveModelFloat
        else:
            return super().select(proc, proc_models)

In [33]:
run_config = CustomRunConfig(select_tag='fixed_pt')
net.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)        
output = output_logger.data.get()
gts = gt_logger.data.get().flatten()[::steps_per_sample]
# net.pause()
net.stop()

## Output accuracy and predictions

In [34]:
gts_arr = gts
output_arr = output.reshape(3,150,8)
output_arr = np.swapaxes(output_arr, 0, 1)

classifier = slayer.classifier.Rate()
prediction = classifier.predict(torch.from_numpy(output_arr)).numpy()
print('Accuracy', np.sum(gts_arr == prediction)/len(prediction))
print(prediction)

Accuracy 0.3333333333333333
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
